------------------------
Notebook: 03_silver_orders_enriched  
Layer   : Silver  
Purpose : Clean, deduplicate, and enrich order facts
Design Decisions:
*Orders are treated as immutable fact records
*Deduplication is applied for idempotency using row_id
*Missing dimension references are retained as UNKNOWN  

------------------------

In [0]:
#Imports for all

from pyspark.sql.functions import (
    col, to_date, round, coalesce, lit, row_number
)
from pyspark.sql.window import Window

In [0]:
##Read Bronze orders
orders_bronze = spark.table("bronze.orders")

In [0]:
#Data Quality Rules:
# - row_id defines the fact grain
# - Business keys must be rpesent to ensure valid joins
#- Quantity <=0 indicates invalid or reversed transactions

orders_dq = (
    orders_bronze
    .filter(col("row_id").isNotNull())
    .filter(col("order_id").isNotNull())
    .filter(col("customer_id").isNotNull())
    .filter(col("product_id").isNotNull())
    .filter(col("quantity") > 0)
)   


In [0]:
#Deterministic deduplication
# Retain teh latest version of each order row based on ingestion date

window_spec =(
    Window.partitionBy("row_id")
    .orderBy(col("ingestion_date").desc())
)
orders_dedup = (
    orders_dq
    .withColumn("rn", row_number().over(window_spec))
    .filter(col("rn") == 1)
    .drop("rn")
)

In [0]:
##Type casting and Projection

orders_clean = (
    orders_dedup
    .select(
        col("row_id"),
        col("order_id"),
        col("customer_id"),
        col("product_id"),
         to_date(col("order_date"), "d/M/yyyy").alias("order_date"),
        to_date(col("ship_date"), "d/M/yyyy").alias("ship_date"),
        col("ship_mode"),
        col("quantity").cast("long").alias("quantity"),
        col("price").cast("double").alias("price"),
        col("discount").cast("double").alias("discount"),
        col("profit").cast("double").alias("profit")
    )
)

In [0]:
##Join with current customer snapshot
# Historical customer attributes are not required at the order grain

customers_current = (
    spark.table("silver.customers_enriched")
    .filter(col("is_current") == True)
    .select(
        "customer_id",
        "customer_name",
        "country"
    )
)

orders_customer_enriched = (
    orders_clean
    .join(customers_current, "customer_id","left")
)

In [0]:
#Join with Products

products_silver = (
    spark.table("silver.products_enriched")
    .select(
        "product_id",
        "category",
        "sub_category"
    )
)

orders_full_enriched = (
    orders_customer_enriched
    .join(products_silver, "product_id","left")
)

In [0]:
#Final Enrichemnt (Profit + Year):
# - Handle orphan facts:
# -- Orders without mayching customer or product refernces are retained and labelled as UNKNOWN to avoid data loss in aggregations

orders_silverdf = (
    orders_full_enriched
    .withColumn("customer_name", coalesce(col("customer_name"), lit("UNKNOWN")))
    .withColumn("country", coalesce(col("country"), lit("UNKNOWN")))
    .withColumn("category", coalesce(col("category"), lit("UNKNOWN")))
    .withColumn("sub_category", coalesce(col("sub_category"), lit("UNKNOWN")))
    .withColumn("profit", round(col("profit"), 2))
    .withColumn("order_year", col("order_date").substr(1, 4).cast("int"))
)

In [0]:
#Write Silver Orders Enriched
orders_silverdf.write.format("delta").mode("overwrite").saveAsTable("silver.orders_enriched")

In [0]:
spark.table("silver.orders_enriched") \
    .filter(
        (col("customer_name") == "UNKNOWN") |
        (col("category") == "UNKNOWN")
    ) \
    .count()

In [0]:
#Validation checks:
#-Ensure schema correctness
#-Verify UNKNOWN handling
#-Sample enriched records

spark.table("silver.orders_enriched").printSchema()
spark.table("silver.orders_enriched").count()   

In [0]:
spark.table("silver.orders_enriched")\
    .select("order_id", "customer_name", "category", "profit")\
    .show(5)

In [0]:
spark.table("silver.orders_enriched") \
    .filter(col("customer_name").isNull() | col("category").isNull()) \
    .count()